# Introducción

El objetivo de este proyecto es generar imágenes realistas de nuevos Pokémon utilizando una red neuronal autoencoder variacional (VAE). Este tipo de red aprende una representación comprimida de las imágenes en un espacio latente, donde cada punto puede corresponder a una posible variante visual. A partir de este espacio, la VAE puede generar nuevos Pokémon basándose en la posición de un punto y sus vecinos cercanos dentro del gráfico del espacio latente, permitiendo crear criaturas visualmente coherentes con aquellas que presentan características similares en el conjunto de entrenamiento.

## Configuración del repositorio

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt
import kagglehub
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# Dataset de Pokemons

Para este proyecto, utilizamos el dataset [Pokémon Images and Types](https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types?resource=download) , que contiene imágenes de Pokémon junto con sus respectivos tipos. Las imágenes están en formato .png, en color RGB, y presentan una resolución adecuada para tareas de generación de imágenes con redes neuronales convolucionales.

Cada entrada del dataset incluye el nombre del Pokémon, su tipo primario y, en algunos casos, un tipo secundario. En este proyecto, nos enfocamos únicamente en el tipo primario como etiqueta condicional para la generación de nuevas imágenes, con el fin de simplificar el problema y aprovechar las correlaciones visuales asociadas a cada tipo.


## Configuración del Dataset

In [ ]:
csv_path = ".." + "/pokemon.csv"
img_path = ".." + "/pokemon"

class PokemonDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.encoded_types = self.label_encoder.fit_transform(self.img_labels.iloc[:, 1])  # Segunda columna: tipo 1

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] + ".png")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.encoded_types[idx]
        return image, label

Se define una transformación que primero redimensiona todas las imágenes a un tamaño fijo de $112 \times 112$ píxeles, lo cual es útil para estandarizar la entrada del modelo, especialmente cuando se trabaja con redes convolucionales que requieren dimensiones consistentes.

Luego, la imagen se convierte a un tensor de PyTorch con valores en el rango $[0, 1]$, utilizando ToTensor(), que también reorganiza las dimensiones de la imagen del formato estándar de PIL (alto, ancho, canales) al formato utilizado por PyTorch (canales, alto, ancho).

Finalmente, se aplica una normalización para escalar los valores de los píxeles al rango $[-1, 1]$

In [ ]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # a [-1, 1]
])

# Conclusión

Se puede concluir que el dataset es altamente adecuado para tareas de generación de imágenes, ya que las muestras presentan un buen nivel de calidad y consistencia visual. Las imágenes están centradas, con los Pokémon recortados sobre fondos transparentes o simples, lo que permite al modelo enfocarse en las características del personaje sin interferencias del entorno.

Además, la estructura del dataset, que incluye etiquetas del tipo primario, permite incorporar información semántica durante el entrenamiento de modelos generativos condicionales. Esta organización facilita la exploración del espacio latente guiado por clases, mejorando la coherencia entre el tipo del Pokémon y su apariencia visual en las imágenes generadas.

Se puede concluir que el dataset es altamente adecuado para tareas de generación de imágenes, ya que las muestras presentan un buen nivel de calidad y consistencia visual. Las imágenes están centradas, con los Pokémon recortados sobre fondos transparentes o simples, lo que permite al modelo enfocarse en las características del personaje sin interferencias del entorno.

Además, la estructura del dataset, que incluye etiquetas del tipo primario, permite incorporar información semántica durante el entrenamiento de modelos generativos condicionales. Esta organización facilita la exploración del espacio latente guiado por clases, mejorando la coherencia entre el tipo del Pokémon y su apariencia visual en las imágenes generadas.